# Libraries

In [23]:
#import libraries

import pandas as pd
import numpy as np
import pandas as pd
import re
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

# Load file

In [24]:
#import file about the french museums infos

Paris_exhibitions=pd.read_csv('Cleaned_data/Paris_exhibitions.csv', sep=',', encoding='UTF8')

pd.set_option('display.max_columns', 20)

# Pre-processing

In [25]:
#gets user input

genre = input("Hi! What is the genre of the exhibition you're interested in?")
audience = input("Are you interested in a Public enfant / Public adolescent & adulte / Tout public / 'Public enfant & adolescent / Public adulte / Public adolescent exhibition?")

#uses user input to print out information
print("You're interested in", genre, "and by an exhibition for", audience)

You're interested in Art contemporain and by an exhibition for Tout public


In [26]:
exhibition_suggestion = Paris_exhibitions.loc[(Paris_exhibitions["genre"] == genre) & (Paris_exhibitions["audience"] == audience)]
exhibition_suggested = exhibition_suggestion.sample()

In [27]:
#retrieve the index number of the exhibition suggested

index_suggested = exhibition_suggested.index.item()

#retrieve the exhibition information

exhibition_suggestion.loc[index_suggested:index_suggested]

,id,url,titre,chapeau,description,date_de_début,date_de_fin,mots_clés,nom_du_lieu,adresse_du_lieu,code_postal,ville,coordonnées_géographiques,type_de_prix,audience,genre,durée
1552,36551,https://www.paris.fr/evenements/spectra-exposi...,Spectra - exposition par Hybrid's Crib,"Hybrid’s Crib, le berceau des nouvelles mixité...",<p><strong>Quelques mots sur l’exposition « Sp...,2023-06-01 17:00:00+02:00,2023-06-10 22:00:00+02:00,"Art contemporain,Expo,Peinture",Galerie Etienne de Causans,25 rue de seine,75006,Paris,"48.8562860194856, 2.33671099056338",gratuit,Tout public,Art contemporain,9


In [28]:
#print out information about the exhibition suggested
print("Here is one exhibition you could like:\n", str(exhibition_suggested['titre']), "\nDescription:", str(exhibition_suggested['chapeau']), "\nPlace:", str(exhibition_suggested['nom_du_lieu']))

Here is one exhibition you could like:
 1552    Spectra - exposition par Hybrid's Crib
Name: titre, dtype: object 
Description: 1552    Hybrid’s Crib, le berceau des nouvelles mixité...
Name: chapeau, dtype: object 
Place: 1552    Galerie Etienne de Causans
Name: nom_du_lieu, dtype: object


# Machine learning

In [29]:
def find_similar_exhibitions(index_suggested):
    df = Paris_exhibitions.drop_duplicates(subset=["chapeau"]).reset_index(drop=True)
    strings = list(df["chapeau"])
    strings_without_digits = []

    # Uses a regex in order to remove the numbers
    pattern = r"\d+"

    for s in strings:
        if isinstance(s, str):
            s_without_digits = re.sub(pattern, "", s)
            strings_without_digits.append(s_without_digits)
        else:
            strings_without_digits.append(s)

    # Convert the keywords into a list
    keywords = list(df["mots_clés"])

    # Create CountVectorizer and fit_transform the texts
    vectorizer = CountVectorizer()
    texts = ["" if pd.isna(text) else text for text in strings_without_digits]
    vectors = vectorizer.fit_transform(texts)
    vocabulary = vectorizer.get_feature_names_out()

    # Create DataFrame with the vectors
    headers = pd.DataFrame.sparse.from_spmatrix(vectors, columns=vocabulary, index=texts)

    # Create CountVectorizer and fit_transform the keywords
    vectorizer = CountVectorizer()
    vectors = vectorizer.fit_transform(keywords)
    vocabulary = vectorizer.get_feature_names_out()

    # Create DataFrame with the vectors
    keywords = pd.DataFrame.sparse.from_spmatrix(vectors, columns=vocabulary, index=keywords)

    # Apply PCA on headers
    pca = PCA(n_components=15)  # Specify the number of components to keep
    vectors_pca = pca.fit_transform(headers.to_numpy())

    # Create DataFrame with PCA components for headers
    headers_pca = pd.DataFrame(vectors_pca, index=df.index)

    # Apply PCA on keywords
    pca = PCA(n_components=15)  # Specify the number of components to keep
    vectors_pca = pca.fit_transform(keywords.to_numpy())

    # Create DataFrame with PCA components for keywords
    keywords_pca = pd.DataFrame(vectors_pca, index=keywords.index)

    # Concatenate the PCA components
    concatenated_df = pd.concat([headers_pca, keywords_pca])

    # Fit Nearest Neighbors model
    neigh = NearestNeighbors(n_neighbors=6)
    neigh.fit(concatenated_df)

    # Find similar exhibitions for a specific index (35 in this case)
    distances, indices = neigh.kneighbors(concatenated_df.iloc[index_suggested:index_suggested+1])

    # Return the similar exhibitions
    return Paris_exhibitions.iloc[indices[0]]

In [30]:
find_similar_exhibitions(index_suggested)

,id,url,titre,chapeau,description,date_de_début,date_de_fin,mots_clés,nom_du_lieu,adresse_du_lieu,code_postal,ville,coordonnées_géographiques,type_de_prix,audience,genre,durée
1552,36551,https://www.paris.fr/evenements/spectra-exposi...,Spectra - exposition par Hybrid's Crib,"Hybrid’s Crib, le berceau des nouvelles mixité...",<p><strong>Quelques mots sur l’exposition « Sp...,2023-06-01 17:00:00+02:00,2023-06-10 22:00:00+02:00,"Art contemporain,Expo,Peinture",Galerie Etienne de Causans,25 rue de seine,75006,Paris,"48.8562860194856, 2.33671099056338",gratuit,Tout public,Art contemporain,9
1389,32279,https://www.paris.fr/evenements/stage-impressi...,STAGE IMPRESSIONS VÉGÉTALES,Stage Impressions végétales grâce à la techniq...,<p>L’objectif est de réaliser un herbier d'emp...,2023-05-24 13:00:00+02:00,2023-05-25 20:00:00+02:00,"Atelier,Nature",Paris-Ateliers Viaduc des Arts,35 avenue Daumesnil,75012,Paris,"48.8478389889225, 2.37461898586463",payant,Public adolescent & adulte,Atelier,1
225,36114,https://www.paris.fr/evenements/albrecht-durer...,"Albrecht Dürer, Vera Molnár, aurèce vettier : ...",AD.VM.AV.IA. De l'autre côté du polyèdre / des...,"<p><em>Vera Molnár, artiste française d'origin...",2023-06-03 22:00:00+02:00,2023-06-04 05:00:00+02:00,Art contemporain,"Transfo, Emmaüs Solidarité",36 rue Jacques Louvel-Tessier,75010,Paris,"48.8715290133243, 2.37135901226259",gratuit,Tout public,Art contemporain,0
1264,36597,https://www.paris.fr/evenements/panic-on-air-3...,Panic on air,Le rendez-vous mensuel de DOA est de retour !\...,<p>Le rendez-vous mensuel de DOA est de retour...,2023-05-24 22:00:00+02:00,2023-05-25 05:00:00+02:00,Clubbing,Panic Room,101 Rue Amelot,75011,Paris,"48.8612128, 2.36764170000004",gratuit,Public adulte,Clubbing,0
669,34196,https://www.paris.fr/evenements/ce-que-le-stre...,Ce que le streaming fait à la musique - Confér...,"A l'occasion de notre discomarché annuel, la m...",<p>Nous avons le plaisir de recevoir le journa...,2023-06-03 19:00:00+02:00,2023-06-03 20:30:00+02:00,"Conférence,Musique",Médiathèque Françoise Sagan,8 rue Léon Schwartzenberg,75010,Paris,"48.8755965903869, 2.35389596112724",gratuit,Public adulte,Conférence,0
1132,34672,https://www.paris.fr/evenements/4211-km-34672,4211 km,Alors qu'en Iran le peuple se révolte depuis p...,<p>4 211 km c’est la distance entre Paris et T...,2023-05-07 23:00:00+02:00,2023-05-31 01:45:00+02:00,Théâtre,Théâtre de Belleville,16 Pass. Piver,75011,Paris,"48.8708999934005, 2.37428102752858",payant,Public adolescent & adulte,Théâtre,23


In [31]:
new_exhibition_suggestion = exhibition_suggestion.drop(index=index_suggested)


# Simulation

In [32]:
def recommend_exhibition(exhibition_suggested):
    index = 0  # Counter to keep track of the suggested exhibitions
    print("Here is the exhibition we found for you:\n")
    print("Title:", exhibition_suggested['titre'].values[0])
    print("Description:", exhibition_suggested['chapeau'].values[0])
    print("Location:", exhibition_suggested['nom_du_lieu'].values[0])
    while True:
        if index < len(exhibition_suggested):
            new_exhibition_suggestion = exhibition_suggested.iloc[index]
            while True:
                response = input("Is the suggested exhibition relevant for you? Yes/No (Y/N): ")
                if response.lower() == "y":
                    similar_exhibitions = find_similar_exhibitions(index_suggested)
                    return pd.DataFrame(similar_exhibitions)
                elif response.lower() == "n":
                    print("Here is another exhibition you could like:\n")
                    new_exhibition_suggestion = exhibition_suggestion.drop(index=index_suggested)
                    new_exhibition_suggested = new_exhibition_suggestion.sample()
                    print("Here is another exhibition you could like:\n")
                    print("Title:", new_exhibition_suggested['titre'].values[0])
                    print("Description:", new_exhibition_suggested['chapeau'].values[0])
                    print("Location:", new_exhibition_suggested['nom_du_lieu'].values[0])
                    response = input("Is the suggested exhibition relevant for you? Yes/No (Y/N): ")
                    if response.lower() == "y":
                        similar_exhibitions = find_similar_exhibitions(index_suggested)
                        return pd.DataFrame(similar_exhibitions)
                    elif response.lower() == "n":
                        new_exhibition_suggestion = exhibition_suggestion.drop(index=index_suggested)
                        new_exhibition_suggested = new_exhibition_suggestion.sample()
                        print("Here is another exhibition you could like:\n")
                        print("Title:", new_exhibition_suggested['titre'].values[0])
                        print("Description:", new_exhibition_suggested['chapeau'].values[0])
                        print("Location:", new_exhibition_suggested['nom_du_lieu'].values[0])
                    break
                else:
                    print("Invalid response. Please enter 'Y' for Yes or 'N' for No.")
        else:
            print("No more exhibitions to suggest.")
            return


# Call the function and pass the exhibition DataFrame
recommend_exhibition(exhibition_suggested)


Here is the exhibition we found for you:

Title: Spectra - exposition par Hybrid's Crib
Description: Hybrid’s Crib, le berceau des nouvelles mixités créatives, présente sa nouvelle exposition intitulée « Spectra » à la galerie Etienne de Causans du 1er au 10 juin.
Location: Galerie Etienne de Causans
Here is another exhibition you could like:

Here is another exhibition you could like:

Title: Solène Moulin-Charnet et Clément Desforges
Description: Habitat habité - Installation de Solène Moulin-Charnet et de Clément Desforge.
Location: Conservatoire à Rayonnement Départemental Iannis-Xenakis


,id,url,titre,chapeau,description,date_de_début,date_de_fin,mots_clés,nom_du_lieu,adresse_du_lieu,code_postal,ville,coordonnées_géographiques,type_de_prix,audience,genre,durée
1552,36551,https://www.paris.fr/evenements/spectra-exposi...,Spectra - exposition par Hybrid's Crib,"Hybrid’s Crib, le berceau des nouvelles mixité...",<p><strong>Quelques mots sur l’exposition « Sp...,2023-06-01 17:00:00+02:00,2023-06-10 22:00:00+02:00,"Art contemporain,Expo,Peinture",Galerie Etienne de Causans,25 rue de seine,75006,Paris,"48.8562860194856, 2.33671099056338",gratuit,Tout public,Art contemporain,9
1389,32279,https://www.paris.fr/evenements/stage-impressi...,STAGE IMPRESSIONS VÉGÉTALES,Stage Impressions végétales grâce à la techniq...,<p>L’objectif est de réaliser un herbier d'emp...,2023-05-24 13:00:00+02:00,2023-05-25 20:00:00+02:00,"Atelier,Nature",Paris-Ateliers Viaduc des Arts,35 avenue Daumesnil,75012,Paris,"48.8478389889225, 2.37461898586463",payant,Public adolescent & adulte,Atelier,1
225,36114,https://www.paris.fr/evenements/albrecht-durer...,"Albrecht Dürer, Vera Molnár, aurèce vettier : ...",AD.VM.AV.IA. De l'autre côté du polyèdre / des...,"<p><em>Vera Molnár, artiste française d'origin...",2023-06-03 22:00:00+02:00,2023-06-04 05:00:00+02:00,Art contemporain,"Transfo, Emmaüs Solidarité",36 rue Jacques Louvel-Tessier,75010,Paris,"48.8715290133243, 2.37135901226259",gratuit,Tout public,Art contemporain,0
1264,36597,https://www.paris.fr/evenements/panic-on-air-3...,Panic on air,Le rendez-vous mensuel de DOA est de retour !\...,<p>Le rendez-vous mensuel de DOA est de retour...,2023-05-24 22:00:00+02:00,2023-05-25 05:00:00+02:00,Clubbing,Panic Room,101 Rue Amelot,75011,Paris,"48.8612128, 2.36764170000004",gratuit,Public adulte,Clubbing,0
669,34196,https://www.paris.fr/evenements/ce-que-le-stre...,Ce que le streaming fait à la musique - Confér...,"A l'occasion de notre discomarché annuel, la m...",<p>Nous avons le plaisir de recevoir le journa...,2023-06-03 19:00:00+02:00,2023-06-03 20:30:00+02:00,"Conférence,Musique",Médiathèque Françoise Sagan,8 rue Léon Schwartzenberg,75010,Paris,"48.8755965903869, 2.35389596112724",gratuit,Public adulte,Conférence,0
1132,34672,https://www.paris.fr/evenements/4211-km-34672,4211 km,Alors qu'en Iran le peuple se révolte depuis p...,<p>4 211 km c’est la distance entre Paris et T...,2023-05-07 23:00:00+02:00,2023-05-31 01:45:00+02:00,Théâtre,Théâtre de Belleville,16 Pass. Piver,75011,Paris,"48.8708999934005, 2.37428102752858",payant,Public adolescent & adulte,Théâtre,23
